In [1]:
!pip install datasets

In [2]:
!pip install polars

In [38]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 12.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━

In [126]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 16.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 25.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 33.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [32]:
import datasets

In [33]:
def count_pieces(fen: str) -> int:
    return sum(c.isalpha() for c in fen.split(" ", 1)[0])

assert count_pieces("QN4n1/6r1/3k4/8/b2K4/8/8/8 b - - 0 1") == 7

def batch_piece_count(batch):
    return {"piece_count": [count_pieces(fen) for fen in batch["fen"]]}

In [44]:
def material_key(fen: str, _PCHR=["K", "Q", "R", "B", "N", "P"]) -> str:
    board = fen.split(" ", 1)[0]
    w = (
        "K" * board.count("K") +
        "Q" * board.count("Q") +
        "R" * board.count("R") +
        "B" * board.count("B") +
        "N" * board.count("N") +
        "P" * board.count("P")
    )
    b = (
        "K" * board.count("k") +
        "Q" * board.count("q") +
        "R" * board.count("r") +
        "B" * board.count("b") +
        "N" * board.count("n") +
        "P" * board.count("p")
    )
    if ((len(w), [_PCHR.index(c) for c in b]) < (len(b), [_PCHR.index(c) for c in w])):
        return b + "v" + w
    else:
        return w + "v" + b

assert material_key("QN4n1/4k1r1/8/8/b2K4/8/8/8 w - - 0 1") == "KRBNvKQN"
assert material_key("7n/8/6N1/1k6/8/3K4/4B3/n7 w - - 0 1") == "KBNvKNN"

def flipped_pawn_bitboards(fen: str) -> (int, int):
    wp = 0
    bp = 0
    x = 0
    for p in fen:
        if p == " ":
            break
        elif p == "P":
            wp |= (1 << x)
            x += 1
        elif p == "p":
            bp |= (1 << x)
            x += 1
        elif p.isdigit():
            x += int(p)
        elif p == "/":
            pass
        else:
            x += 1
    return wp, bp

assert flipped_pawn_bitboards("rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1") == (67272588153323520, 65280)

def op1(fen: str) -> bool:
    wp, bp = flipped_pawn_bitboards(fen)
    return bool(
        (bp << 8 * 1) & wp or
        (bp << 8 * 2) & wp or
        (bp << 8 * 3) & wp or
        (bp << 8 * 4) & wp or
        (bp << 8 * 5) & wp)

assert op1("rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1")
assert not op1("7n/8/6N1/1k6/8/3K4/4B2P/n7 w - - 0 1")
assert op1("7n/7p/6N1/1k6/8/3K4/4B2P/n7 w - - 0 1")
assert not op1("7n/6p1/6N1/1k6/8/3K4/4B2P/n7 w - - 0 1")
assert op1("7n/8/6N1/1k4p1/6P1/3K4/4B2P/n7 w - - 0 1")
assert not op1("7n/6P1/6N1/1k4p1/8/3K4/4B2P/n7 w - - 0 1")

def batch_augment(batch):
    return {
        "material": [material_key(fen) for fen in batch["fen"]],
        "op1": [op1(fen) for fen in batch["fen"]],
    }

In [55]:
ds = datasets.load_dataset("nfiekas/fishnet-evals", split="train", streaming=True)
ds = ds.remove_columns(["move"])
ds = ds.map(batch_piece_count, batched=True, batch_size=10_000)
ds = ds.filter(lambda x: x["piece_count"] <= 8)
ds = ds.map(batch_augment, batched=True, batch_size=10_000)

Resolving data files:   0%|          | 0/144 [00:00<?, ?it/s]

In [56]:
next(ds)

{'fen': '8/7p/6pk/2pr4/8/2KP4/4R3/8 w - - 0 43',
 'cp': -259,
 'mate': None,
 'piece_count': 8,
 'material': 'KRPPPvKRP',
 'op1': False}

In [57]:
import polars as pl

In [58]:
import numpy as np

In [59]:
PIECE_TO_PLANE = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11,
}

def fen_to_onehot(fen: str) -> np.ndarray:
    board, turn, _ = fen.split(" ", 2)
    board_tensor = np.zeros((12, 8, 8), dtype=np.uint8)
    for (y, rank) in enumerate(board.split("/")):
        x = 0
        for ch in rank:
            if ch.isdigit():
                x += int(ch)
            else:
                board_tensor[PIECE_TO_PLANE[ch], y, x] = 1
                x += 1
    flat_tensor = board_tensor.reshape(-1)
    turn_tensor = np.array([int(turn == "w")], dtype=np.uint8)
    return np.concatenate([flat_tensor, turn_tensor])

In [60]:
fen_to_onehot("4k3/8/8/8/8/8/8/3K4 w - - 0 1")

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [61]:
import torch.utils.data
import torch

In [63]:
class EndgameDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_tensor = torch.tensor(fen_to_onehot(self.df["fen"][idx]), dtype=torch.float32)

        mate = self.df["mate"][idx]
        cp = self.df["cp"][idx]
        label_tensor = torch.tensor(int(mate is not None and mate > 0 or cp is not None and cp > 0), dtype=torch.float32)
        
        return input_tensor, label_tensor

In [121]:
full_dataset = EndgameDataset(df)

debug_size = 10_000
val_size = 100_000
train_dataset, debug_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [len(full_dataset) - debug_size - val_size, debug_size, val_size])

In [149]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10_000, shuffle=True, num_workers=30)
debug_loader = torch.utils.data.DataLoader(debug_dataset, batch_size=1000, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=10_000, shuffle=False, num_workers=10)

In [134]:
import torch.nn as nn
import lightning
import lightning.pytorch

In [147]:
def material_count_predict(x):
    balance = 0
    for i, v in enumerate([1, 3, 3, 5, 9, 0, -1, -3, -3, -5, -9, 0]):
        balance += sum(x[64 * i:64 * (i + 1)]) * v
    return int(balance >= 0)

def material_count_predict_batch(inputs):
    return torch.tensor([material_count_predict(x) for x in inputs], dtype=torch.int, device=inputs.device).unsqueeze(1)

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in debug_loader:
        preds = material_count_predict_batch(inputs)
        correct += (preds.squeeze() == labels).sum().item()
        total += len(labels)

accuracy = correct / total
print(f"Material count prediction acc: {accuracy:.4f}")

Material count prediction acc: 0.7568


In [176]:
class EndgameNet(lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(64 * 6 * 2 + 1, 32),
            nn.ReLU(),
            nn.Linear(32, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
        )
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y.unsqueeze(1).float())
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y.unsqueeze(1).float())
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds == y.unsqueeze(1)).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [183]:
model = EndgameNet()
trainer = lightning.pytorch.Trainer(max_epochs=10, accelerator="auto")
trainer.fit(model, train_loader, val_loader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type              | Params | Mode 
------------------------------------------------------
0 | model   | Sequential        | 24.9 K | train
1 | loss_fn | BCEWithLogitsLoss | 0      | train
------------------------------------------------------
24.9 K    Trainable params
0         Non-trainable params
24.9 K    Total params
0.100     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                          | 0/? [00:00<?, ?it/s]

Training: |                                                                 | 0/? [00:00<?, ?it/s]

Validation: |                                                               | 0/? [00:00<?, ?it/s]

Validation: |                                                               | 0/? [00:00<?, ?it/s]

...

Validation: |                                                               | 0/? [00:00<?, ?it/s]

Validation: |                                                               | 0/? [00:00<?, ?it/s]

Validation: |                                                               | 0/? [00:00<?, ?it/s]

Validation: |                                                               | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [184]:
import timeit
import torch

model = model.to(torch.device("cpu"))

quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

trainer.validate(model=quantized_model, dataloaders=val_loader)

Validation: |                                                               | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.8858199715614319
        val_loss            0.2653074264526367
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.2653074264526367, 'val_acc': 0.8858199715614319}]

In [186]:
inputs, _ = next(iter(debug_loader))
inputs = inputs[:30]
inputs = inputs.to(torch.device("cpu"))

traced_model = torch.jit.trace(quantized_model, inputs)

def run_inference():
    traced_model(inputs)

with torch.no_grad():
    num_runs = 100_000
    for _ in range(num_runs):
        run_inference()
    avg_time_us = (timeit.timeit(run_inference, number=num_runs) / num_runs) * 1e6
    print(f"Avg inference time with quantization for batch size 1: {avg_time_us:.2f} µs")

Avg inference time with quantization for batch size 1: 74.48 µs
